In [192]:
import numpy as np
import tensorflow.compat.v1 as tf

In [193]:
tf.disable_eager_execution()

In [194]:
def get_dataset(batch_size=100):
    """
    注意：关于 dataset 的操作都会返回一个新的 dataset, 另忘记用新的变量去接收。
    """
    mnist = tf.keras.datasets.mnist
    train, test = mnist.load_data()
    train_dataset = tf.data.Dataset.from_tensor_slices(train)
    train_dataset = train_dataset.map(lambda x,y: (tf.reshape(x, [784,]), tf.one_hot(y, 10)))
    train_dataset = train_dataset.repeat(100).batch(batch_size)
    
    test_dataset = tf.data.Dataset.from_tensor_slices(test)
    test_dataset = test_dataset.map(lambda x,y: (tf.reshape(x, [784,]), tf.one_hot(y, 10)))
    test_dataset = test_dataset.batch(100)

    return train_dataset, test_dataset


In [208]:
learning_rate = 0.01
epochs = 10
batch_size = 100
n_samples = 60000

In [196]:
def model(graph):
    with graph.as_default():
        x = tf.placeholder(dtype=tf.float32,
                          shape=(None, 784), name='x')
        y = tf.placeholder(dtype=tf.float32,
                          shape=(None, 10), name='y')
        
        with tf.variable_scope('LR'):
            w = tf.Variable(tf.zeros([784, 10]))
            b = tf.Variable(tf.zeros([10]))
            
            pred = tf.nn.softmax(tf.matmul(x, w) + b)
            
            # 用 tf.clip_by_value 防止计算时出现  inf 
            loss = tf.reduce_mean(
                -tf.reduce_sum(y * tf.log(tf.clip_by_value(pred,1e-10,1.0)), reduction_indices=1)
            )
    return x, y, loss, pred
    

In [209]:
def train():
    g1 = tf.Graph()
    with g1.as_default():
        train_dataset, test_dataset = get_dataset(batch_size)
        train_iterator = tf.data.make_one_shot_iterator(train_dataset)
        train_next_element = train_iterator.get_next()
        
        test_iterator = tf.data.make_one_shot_iterator(test_dataset)
        test_next_element = test_iterator.get_next()
        
        x, y, loss, pred = model(g1)
        
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
        
        init = tf.global_variables_initializer()
        
        with tf.Session() as sess:
            tf.summary.FileWriter('./logs/logistic-reg-mnist', sess.graph)
            
            sess.run(init)
            
            n_batches = int(n_samples / batch_size)
            print('Start:')
            for epoch in range(epochs):
                avg_loss = 0
                for batch in range(n_batches):
                    x_train, y_train = sess.run(train_next_element)
                    feed_dict = {x: x_train, y: y_train}
                    loss_val, _ = sess.run([loss, optimizer], feed_dict)
                    avg_loss += loss_val / n_samples
                print('Epoch ', epoch + 1, " : loss = ", avg_loss)
            
            # 测试 accuracy
            acc = 0
            total_test = 0
            try:
#                 while True:
                    x_test, y_test = sess.run(test_next_element)
                    feed_dict = {x: x_test}
                    total_test += x_test.shape[0]
                    y_pred = sess.run(pred, feed_dict)
                    p_idx = tf.argmax(y_pred, 1)
                    t_idx = tf.argmax(y_test, 1)
                    equal_idx = tf.equal(p_idx, t_idx)
                    equals = tf.cast(equal_idx, dtype=tf.float32)
                    acc += sess.run(tf.reduce_sum(equals))
            except tf.errors.OutOfRangeError:
                print("Iterator Finished")
        print('Last loss:', avg_loss) 
        print('total_test:', total_test)
        print('accuracy:', (acc / total_test) * 100, '%') 

In [210]:
train()

Start:
Epoch  1  : loss =  0.0892284685492515
Epoch  2  : loss =  0.06995963064034778
Epoch  3  : loss =  0.064606470644474
Epoch  4  : loss =  0.06359267535209655
Epoch  5  : loss =  0.061362683415412886
Epoch  6  : loss =  0.06108994638522466
Epoch  7  : loss =  0.059992963830630026
Epoch  8  : loss =  0.059530436972777004
Epoch  9  : loss =  0.0592103311181068
Epoch  10  : loss =  0.05853665634393688
Last loss: 0.05853665634393688
total_test: 100
accuracy: 83.0 %


这个例子，完整的说明了 tf 1.0 怎样去训练，测试。

1. 定义模型
2. 数据集处理

<img src="Logistic-Regression-Mnist.png" />